**01. 데이터 로딩**

In [1]:
# 라이브러리
import numpy as np
import pandas as pd

# 데이터 로딩
dataset = pd.read_csv('./final_dataset.csv')

# 불필요한 열 제거
del dataset['Unnamed: 0']

# 데이터 보기
dataset

,open,high,low,close,volume,score,index,probability,l1,l2,l3,l4,lgap,lrate,open_high,open_low,open_close,zone
0,-4.33,-2.76,-2.76,-1.02,12.59,1.33,0.89,47.11,-1.92,-1.90,-1.89,-1.87,6.29,6.0,2.0,-2.0,0.0,+3 ~ +10
1,1.34,0.41,-8.52,-6.68,13.34,1.64,0.67,27.96,1.20,-3.11,-7.59,-12.24,7.50,20.0,1.0,-12.0,-8.0,-3 ~ -1
2,-11.08,-4.34,0.57,3.52,12.72,1.60,0.63,23.64,-2.92,-0.58,2.08,5.13,7.00,12.0,8.0,0.0,7.0,-3 ~ -1
3,5.26,3.06,5.26,3.83,12.55,1.60,0.63,23.64,2.45,3.44,4.52,5.68,6.73,9.0,6.0,0.0,6.0,-1 ~ +1
4,6.95,3.38,5.32,-0.10,12.77,1.60,0.63,23.64,0.80,1.64,2.53,3.48,6.40,6.0,3.0,-2.0,-1.0,-3 ~ -1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1901794,-0.37,-0.47,-0.17,0.34,6.99,7.96,2.40,79.01,-0.22,-0.07,0.08,0.24,5.28,1.0,1.0,0.0,1.0,-1 ~ +1
1901795,0.68,0.84,0.27,0.00,8.27,7.91,3.04,79.75,0.70,0.42,0.14,-0.15,5.78,2.0,1.0,-0.0,0.0,-1 ~ +1
1901796,-1.04,-1.07,-0.64,-0.03,7.44,7.91,3.04,79.75,-0.76,-0.55,-0.34,-0.12,5.42,2.0,1.0,0.0,1.0,+1 ~ +3
1901797,-0.31,-1.11,-0.92,-1.48,6.19,7.90,1.95,77.30,-1.39,-1.30,-1.20,-1.11,5.20,1.0,0.0,-1.0,-0.0,-3 ~ -1


**02. Train 데이터와 Test 데이터 분리**

In [2]:
# 구역 숫자로 변환
def zone(x) :
    if x == '-30 ~ -10' : return 0
    elif x == '-10 ~ -3' : return 1
    elif x == '-3 ~ -1' : return 2
    elif x == '-1 ~ +1' : return 3
    elif x == '+1 ~ +3' : return 4
    elif x == '+3 ~ +10' : return 5
    elif x == '+10 ~ +20' : return 6
    else : return 7
        
dataset['zone'] = dataset['zone'].apply(lambda x : zone(x))

# 어레이로 데이터 변경
dataset = np.array(dataset)

# 80%를 학습하는데 이용
train_idx = int(len(dataset) * 0.8)
train_X, train_Y = dataset[:train_idx, :-1], dataset[:train_idx, -1]
test_X, test_Y = dataset[train_idx:, :-1], dataset[train_idx:, -1]

# 카테고리 지정
import tensorflow as tf
train_Y = tf.keras.utils.to_categorical(train_Y, num_classes=8)
test_Y = tf.keras.utils.to_categorical(test_Y, num_classes=8)

**03. Model Build**

In [3]:
model = tf.keras.Sequential([
                             tf.keras.layers.Dense(units=32,activation='relu',input_shape=(train_X.shape[1],)),
                             tf.keras.layers.Dense(units=16,activation='relu'),
                             tf.keras.layers.Dense(units=8, activation='softmax')
])
        
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.07), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                576       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
Total params: 1,240
Trainable params: 1,240
Non-trainable params: 0
_________________________________________________________________


**04. Learning**

In [4]:
history = model.fit(train_X[::902], train_Y[::902], epochs=25, batch_size=4096, validation_split=0.25)

Train on 1265 samples, validate on 422 samples
Epoch 1/25
1265/1265 [==============================] - 0s 349us/sample - loss: 17.3125 - accuracy: 0.0158 - val_loss: 6.2530 - val_accuracy: 0.2156
Epoch 2/25
1265/1265 [==============================] - 0s 9us/sample - loss: 6.1606 - accuracy: 0.2435 - val_loss: 3.1810 - val_accuracy: 0.4858
Epoch 3/25
1265/1265 [==============================] - 0s 9us/sample - loss: 3.0113 - accuracy: 0.4617 - val_loss: 2.4453 - val_accuracy: 0.3009
Epoch 4/25
1265/1265 [==============================] - 0s 9us/sample - loss: 2.3647 - accuracy: 0.2846 - val_loss: 2.0203 - val_accuracy: 0.2773
Epoch 5/25
1265/1265 [==============================] - 0s 8us/sample - loss: 1.9576 - accuracy: 0.2814 - val_loss: 1.8630 - val_accuracy: 0.4265
Epoch 6/25
1265/1265 [==============================] - 0s 9us/sample - loss: 1.8071 - accuracy: 0.4340 - val_loss: 1.6770 - val_accuracy: 0.2654
Epoch 7/25
1265/1265 [==============================] - 0s 8us/sample - lo

**05. Test Evaluate**

In [5]:
model.evaluate(test_X, test_Y)

380360/380360 [==============================] - 9s 22us/sample - loss: 1.2885 - accuracy: 0.5429


[1.288546987895723, 0.5428778]